In [340]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import functions as fun
from linearmodels.iv import IV2SLS
import statsmodels.formula.api as sm

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

In [341]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
9,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
10,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
21,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
28,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
29,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
30,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
31,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
32,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
41,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [342]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
1,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
2,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
3,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
4,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
5,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
6,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
7,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
8,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
9,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [343]:
fun.BLP(data, 'Range')
fun.BLP(data, 'HP')
fun.BLP(data, 'Chargetime')
fun.GH(data, 'Range', 0.5)
fun.GH(data, 'HP', 0.5)
fun.GH(data, 'Chargetime', 0.5)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,Range_BLP,HP_BLP,Chargetime_BLP,Range_GH,HP_GH,Chargetime_GH
0,1,2021,1.037e-02,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0,21513,13583,1861,14999,7405,1317
1,1,2022,5.976e-03,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0,32180,20339,2664,21535,10602,1796
2,1,2023,2.860e-03,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0,41774,26403,3241,27923,15010,2099
3,2,2023,4.848e-05,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0,41769,26390,3241,27468,14562,2099
4,3,2019,4.063e-02,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0,5809,3901,570,4709,3105,533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,189,2023,6.464e-05,Volvo,EX30,475,368245.000,268,28,SUV,B,SE,4,1.0,41699,26336,3247,21894,17776,1813
330,190,2021,1.465e-02,Volvo,XC40,457,462060.600,402,28,SUV,C,SE,363,1.0,21456,13382,1867,11699,10265,1138
331,190,2022,3.331e-02,Volvo,XC40,457,416263.400,402,28,SUV,C,SE,1020,1.0,32123,20138,2670,17028,16217,1483
332,190,2023,3.175e-02,Volvo,XC40,457,439266.600,402,28,SUV,C,SE,1965,1.0,41717,26202,3247,22067,19786,1813


In [344]:
product_data = data.rename(columns={
    'Year': 'market_ids',
    'Model': 'product_ids',
    'Market_share': 'shares',
    'Price': 'prices',
    'Manufacturer': 'firm_ids',
    'Segment': 'nesting_ids',
})

In [345]:
#Scale for better intepretation
product_data['prices'] = product_data['prices']/10_000 #(Change in ms(%) for change in pris in 10.000)
product_data['HP'] = product_data['HP']/10           #(Change in ms(%) for change in HP in 10)
product_data['Range'] = product_data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [346]:
# Creating dummy for china
product_data['China'] = (product_data['Country'] == 'CN').astype(int)

# Outside share

In [347]:
product_data.loc[product_data['market_ids'] == 2013, 'shares'] = product_data.loc[product_data['market_ids'] == 2013, 'Sales'] / 180632
product_data.loc[product_data['market_ids'] == 2014, 'shares'] = product_data.loc[product_data['market_ids'] == 2014, 'Sales'] / 188406
product_data.loc[product_data['market_ids'] == 2015, 'shares'] = product_data.loc[product_data['market_ids'] == 2015, 'Sales'] / 206653
product_data.loc[product_data['market_ids'] == 2016, 'shares'] = product_data.loc[product_data['market_ids'] == 2016, 'Sales'] / 222471
product_data.loc[product_data['market_ids'] == 2017, 'shares'] = product_data.loc[product_data['market_ids'] == 2017, 'Sales'] / 221471
product_data.loc[product_data['market_ids'] == 2018, 'shares'] = product_data.loc[product_data['market_ids'] == 2018, 'Sales'] / 252328
product_data.loc[product_data['market_ids'] == 2019, 'shares'] = product_data.loc[product_data['market_ids'] == 2019, 'Sales'] / 258727
product_data.loc[product_data['market_ids'] == 2020, 'shares'] = product_data.loc[product_data['market_ids'] == 2020, 'Sales'] / 230060
product_data.loc[product_data['market_ids'] == 2021, 'shares'] = product_data.loc[product_data['market_ids'] == 2021, 'Sales'] / 222210
product_data.loc[product_data['market_ids'] == 2022, 'shares'] = product_data.loc[product_data['market_ids'] == 2022, 'Sales'] / 181030
product_data.loc[product_data['market_ids'] == 2023, 'shares'] = product_data.loc[product_data['market_ids'] == 2023, 'Sales'] / 203690

In [348]:
product_data['outside_share'] = 1 - product_data.groupby('market_ids')['shares'].transform('sum')
product_data[['shares', 'outside_share']].describe()

,shares,outside_share
count,3.340e+02,334.000
mean,2.128e-03,0.841
std,5.618e-03,0.109
min,4.347e-06,0.696
25%,1.227e-04,0.696
50%,7.149e-04,0.831
75%,2.161e-03,0.938
max,8.814e-02,0.998


# Pure Logit Demand OLS

In [349]:
OLS = sm.ols('np.log(shares/outside_share) ~ prices + Range + HP + Chargetime + C(firm_ids) + C(market_ids)', product_data).fit(cov_type='HC0')
OLS.summary()#.tables[1]

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 54, but rank is 49
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     np.log(shares / outside_share)   R-squared:                       0.508
Model:                                        OLS   Adj. R-squared:                  0.413
Method:                             Least Squares   F-statistic:                     272.8
Date:                            Wed, 05 Jun 2024   Prob (F-statistic):          1.80e-208
Time:                                    01:31:27   Log-Likelihood:                -594.31
No. Observations:                             334   AIC:                             1299.
Df Residuals:                                 279   BIC:                             1508.
Df Model:                                      54                                         
Covariance Type:                              HC0                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      -12.2167      1.445     -8.452      0.000     -15.050      -9.384
C(firm_ids)[T.Audi]              3.1326      0.939      3.336      0.001       1.292       4.973
C(firm_ids)[T.BMW]               1.8139      0.951      1.908      0.056      -0.050       3.677
C(firm_ids)[T.BYD]              -1.3255      1.072     -1.237      0.216      -3.426       0.775
C(firm_ids)[T.Citroen]           0.3291      0.964      0.341      0.733      -1.560       2.219
C(firm_ids)[T.Cupra]             1.6711      0.958      1.744      0.081      -0.207       3.549
C(firm_ids)[T.DS]               -0.3129      0.936     -0.334      0.738      -2.146       1.521
C(firm_ids)[T.Dacia]             2.1581      0.924      2.335      0.020       0.347       3.970
C(firm_ids)[T.Fiat]              1.7187      0.998      1.723      0.085      -0.237       3.674
C(firm_ids)[T.Fisker]           -0.7517      1.026     -0.733      0.464      -2.762       1.259
C(firm_ids)[T.Ford]              0.5733      1.806      0.318      0.751      -2.966       4.112
C(firm_ids)[T.Honda]             0.3437      1.017      0.338      0.735      -1.650       2.337
C(firm_ids)[T.Hongqi]            1.2803      0.951      1.346      0.178      -0.584       3.145
C(firm_ids)[T.Hyundai]           1.3215      0.914      1.445      0.148      -0.471       3.114
C(firm_ids)[T.JAC]              -0.3696      0.878     -0.421      0.674      -2.091       1.351
C(firm_ids)[T.Jaguar]            0.8504      1.016      0.837      0.403      -1.141       2.842
C(firm_ids)[T.Jeep]              0.1567      0.887      0.177      0.860      -1.582       1.896
C(firm_ids)[T.Kia]              -0.0057      0.964     -0.006      0.995      -1.895       1.884
C(firm_ids)[T.Lexus]             0.1636      1.048      0.156      0.876      -1.890       2.217
C(firm_ids)[T.MG]                0.9368      0.914      1.025      0.305      -0.855       2.728
C(firm_ids)[T.Maxus]             0.5493      1.156      0.475      0.635      -1.716       2.815
C(firm_ids)[T.Mazda]             2.9140      0.969      3.007      0.003       1.015       4.813
C(firm_ids)[T.Mercedes-Benz]     2.4201      0.916      2.641      0.008       0.624       4.216
C(firm_ids)[T.Mini]              2.5065      1.004      2.497      0.013       0.539       4.474
C(firm_ids)[T.Nio]              -1.6574      0.986     -1.681      0.093      -3.590       0.275
C(firm_ids)[T.Nissan]            2.3661      0.914      2.589      0.010       0.575       4.158
C(firm_ids)[T.Opel]              0.5618      0.946      0.594      0.553      -1.293       2.416
C(firm_ids)[T.Peugeot]           0.8650  

### Inelastic Demands

In [350]:
product_data['demand_instruments0'] = product_data['prices']
ols = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)+C(market_ids)')
ols_problem = pyblp.Problem(ols, product_data)
ols_results = ols_problem.solve(rho=0)
total_count = 0
for i in range(2013,2024):
    ols_elasticities = ols_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(ols_elasticities).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 29


# Pure Logit BLP Demand

In [351]:
formula = 'np.log(shares/outside_share) ~ [prices ~ Range_BLP + HP_BLP + Chargetime_BLP] + Range + HP + Chargetime + C(firm_ids)'
BLP = IV2SLS.from_formula(formula, product_data).fit(cov_type='robust')
BLP.first_stage

,prices
R-squared,0.7932
Partial R-squared,0.0134
Shea's R-squared,0.0134
Partial F-statistic,7.9860
P-value (Partial F-stat),0.0463
Partial F-stat Distn,chi2(3)
==============================,===========
Range,0.5010
,(2.1244)
HP,1.4899


In [352]:
BLP.summary#.tables[1]

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                                IV-2SLS Estimation Summary                                
==========================================================================================
Dep. Variable:     np.log(shares / outside_share)   R-squared:                      0.2029
Estimator:                                IV-2SLS   Adj. R-squared:                 0.0815
No. Observations:                             334   F-statistic:                 8.231e+14
Date:                            Wed, Jun 05 2024   P-value (F-stat)                0.0000
Time:                                    01:31:27   Distribution:                 chi2(45)
Cov. Estimator:                            robust                                         
                                                                                          
                                      Parameter Estimates                                       
================================================================================================
                              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------------------
Range                            0.0858     0.0433     1.9843     0.0472      0.0011      0.1706
HP                              -0.1043     0.0700    -1.4896     0.1363     -0.2416      0.0329
Chargetime                      -0.0441     0.0176    -2.5141     0.0119     -0.0785     -0.0097
C(firm_ids)[T.Aiways]           -8.4899     2.1056    -4.0321     0.0001     -12.617     -4.3631
C(firm_ids)[T.Audi]             -7.8884     1.1996    -6.5757     0.0000     -10.240     -5.5372
C(firm_ids)[T.BMW]              -8.7514     1.0581    -8.2707     0.0000     -10.825     -6.6775
C(firm_ids)[T.BYD]              -8.5820     2.6867    -3.1942     0.0014     -13.848     -3.3161
C(firm_ids)[T.Citroen]          -9.6682     1.0984    -8.8020     0.0000     -11.821     -7.5154
C(firm_ids)[T.Cupra]            -6.8495     2.0696    -3.3095     0.0009     -10.906     -2.7931
C(firm_ids)[T.DS]               -9.8655     1.2047    -8.1893     0.0000     -12.227     -7.5043
C(firm_ids)[T.Dacia]            -7.1934     1.1730    -6.1325     0.0000     -9.4924     -4.8944
C(firm_ids)[T.Fiat]             -8.0094     1.1384    -7.0360     0.0000     -10.241     -5.7783
C(firm_ids)[T.Fisker]           -6.7039     3.9009    -1.7186     0.0857     -14.350      0.9417
C(firm_ids)[T.Ford]             -7.2730     3.0441    -2.3892     0.0169     -13.239     -1.3066
C(firm_ids)[T.Honda]            -8.9130     1.3310    -6.6966     0.0000     -11.522     -6.3044
C(firm_ids)[T.Hongqi]           -6.4426     2.4027    -2.6813     0.0073     -11.152     -1.7333
C(firm_ids)[T.Hyundai]          -8.0377     1.6409    -4.8983     0.0000     -11.254     -4.8216
C(firm_ids)[T.JAC]              -8.6208     2.0663    -4.1720     0.0000     -12.671     -4.5708
C(firm_ids)[T.Jaguar]           -9.0901     1.6241    -5.5969     0.0000     -12.273     -5.9069
C(firm_ids)[T.Jeep]             -9.2003     1.3665    -6.7328     0.0000     -11.879     -6.5221
C(firm_ids)[T.Kia]              -8.7262     2.0852    -4.1848     0.0000     -12.813     -4.6393
C(firm_ids)[T.Lexus]            -8.8154     1.9179    -4.5963     0.0000     -12.574     -5.0563
C(firm_ids)[T.MG]               -7.6357     1.8235    -4.1873     0.0000     -11.210     -4.0616
C(firm_ids)[T.Maxus]            -8.8114     1.3903    -6.3378     0.0000     -11.536     -6.0865
C(firm_ids)[T.Mazda]            -6.4149     1.0990    -5.8373     0.0000     -8.5688     -4.2610
C(firm_ids)[T.Mercedes-Benz]    -9.4087     1.5115    -6.2247     0.0000     -12.371     -6.4462
C(firm_ids)[T.Mini]             -7.2539     1.0492    -6.9139     0.0000     -9.3102     -5.1975
C(firm_ids)[T.Nio]              -8.7254     2.8733    -3.0367     0.0024     -14.357     -3.0938
C(firm_ids)[T.Nissan]  

### Inelastic Demands

In [353]:
product_data['demand_instruments0'] = product_data['Range_BLP']
product_data['demand_instruments1'] = product_data['HP_BLP']
product_data['demand_instruments2'] = product_data['Chargetime_BLP']
blp = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)')
blp_problem = pyblp.Problem(blp, product_data)
blp_results = blp_problem.solve(rho=0)
total_count = 0
for i in range(2013,2024):
    blp_elasticities = blp_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(blp_elasticities).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 229


# Pure Logit GH Demand

In [354]:
formula = 'np.log(shares/outside_share) ~ 0 + [prices ~ Range_GH + HP_GH + Chargetime_GH] + Range + HP + Chargetime + C(firm_ids) + C(market_ids)'
GH = IV2SLS.from_formula(formula, product_data).fit(cov_type='robust')
GH.first_stage

,prices
R-squared,0.8293
Partial R-squared,0.1627
Shea's R-squared,0.1627
Partial F-statistic,17.405
P-value (Partial F-stat),0.0006
Partial F-stat Distn,chi2(3)
==============================,===========
Range,0.6827
,(3.0607)
HP,1.3295


In [355]:
GH.summary#.tables[1]

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                                IV-2SLS Estimation Summary                                
==========================================================================================
Dep. Variable:     np.log(shares / outside_share)   R-squared:                      0.2875
Estimator:                                IV-2SLS   Adj. R-squared:                 0.1496
No. Observations:                             334   F-statistic:                -3.647e+16
Date:                            Wed, Jun 05 2024   P-value (F-stat)                1.0000
Time:                                    01:31:27   Distribution:                 chi2(55)
Cov. Estimator:                            robust                                         
                                                                                          
                                      Parameter Estimates                                       
================================================================================================
                              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------------------
Range                            0.1651     0.0249     6.6226     0.0000      0.1163      0.2140
HP                               0.0906     0.0360     2.5153     0.0119      0.0200      0.1612
Chargetime                      -0.0143     0.0133    -1.0806     0.2799     -0.0403      0.0117
C(firm_ids)[T.Aiways]           -14.770     1.6985    -8.6961     0.0000     -18.099     -11.441
C(firm_ids)[T.Audi]             -9.4124     1.6332    -5.7630     0.0000     -12.613     -6.2113
C(firm_ids)[T.BMW]              -11.312     1.4867    -7.6088     0.0000     -14.226     -8.3979
C(firm_ids)[T.BYD]              -17.196     1.9933    -8.6268     0.0000     -21.103     -13.289
C(firm_ids)[T.Citroen]          -12.992     1.2978    -10.011     0.0000     -15.535     -10.448
C(firm_ids)[T.Cupra]            -13.210     1.6272    -8.1184     0.0000     -16.399     -10.021
C(firm_ids)[T.DS]               -14.096     1.3516    -10.429     0.0000     -16.746     -11.447
C(firm_ids)[T.Dacia]            -11.634     1.2871    -9.0392     0.0000     -14.157     -9.1115
C(firm_ids)[T.Fiat]             -11.882     1.3467    -8.8226     0.0000     -14.521     -9.2421
C(firm_ids)[T.Fisker]           -18.069     2.2168    -8.1511     0.0000     -22.414     -13.724
C(firm_ids)[T.Ford]             -14.918     2.4377    -6.1198     0.0000     -19.696     -10.140
C(firm_ids)[T.Honda]            -13.489     1.4145    -9.5364     0.0000     -16.261     -10.716
C(firm_ids)[T.Hongqi]           -14.010     1.7716    -7.9083     0.0000     -17.482     -10.538
C(firm_ids)[T.Hyundai]          -13.105     1.5281    -8.5757     0.0000     -16.100     -10.110
C(firm_ids)[T.JAC]              -15.318     1.5871    -9.6516     0.0000     -18.428     -12.207
C(firm_ids)[T.Jaguar]           -12.896     1.7618    -7.3200     0.0000     -16.349     -9.4431
C(firm_ids)[T.Jeep]             -13.752     1.3887    -9.9023     0.0000     -16.474     -11.030
C(firm_ids)[T.Kia]              -14.922     1.6931    -8.8133     0.0000     -18.241     -11.604
C(firm_ids)[T.Lexus]            -13.981     1.7205    -8.1259     0.0000     -17.353     -10.609
C(firm_ids)[T.MG]               -13.724     1.5372    -8.9281     0.0000     -16.737     -10.712
C(firm_ids)[T.Maxus]            -13.287     1.4983    -8.8682     0.0000     -16.224     -10.351
C(firm_ids)[T.Mazda]            -10.826     1.2585    -8.6024     0.0000     -13.293     -8.3597
C(firm_ids)[T.Mercedes-Benz]    -9.4172     1.8568    -5.0718     0.0000     -13.057     -5.7780
C(firm_ids)[T.Mini]             -10.854     1.2028    -9.0242     0.0000     -13.212     -8.4968
C(firm_ids)[T.Nio]              -17.731     1.8891    -9.3858     0.0000     -21.434     -14.028
C(firm_ids)[T.Nissan]  

### Inelastic Demands

In [356]:
product_data['demand_instruments0'] = product_data['Range_GH']
product_data['demand_instruments1'] = product_data['HP_GH']
product_data['demand_instruments2'] = product_data['Chargetime_GH']
gh = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)+C(market_ids)')
gh_problem = pyblp.Problem(gh, product_data)
gh_results = gh_problem.solve(rho=0)
total_count = 0
for i in range(2013,2024):
    gh_elasticities = gh_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(gh_elasticities).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 0


## Pure Logit GH Price-Elasticities

In [357]:
logit = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)+C(market_ids)')
logit_problem = pyblp.Problem(logit, product_data)
logit_results = logit_problem.solve(rho=0)
data2023 = product_data[product_data['market_ids'] == 2023]
logit_elasticities = logit_results.compute_elasticities(market_id=2023)
pd.DataFrame(logit_elasticities, index = data2023['product_ids'], columns=data2023['product_ids'])

product_ids,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,iX,iX1,iX3,Atto 3,Dolphin,Han,Seal,Tang,Berlingo,C4,ë-C4 X,e-SpaceTourer,Jumpy,Born,Spring,...,Taycan,Megane,Zoe,Enyaq iV,Fortwo,Korando,Solterra,Model 3,Model S,Model X,Model Y,bZ4X,Proace City Verso,Proace Verso,Golf,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
product_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
U5,-2.959,5.947e-05,0.003,0.002,0.085,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,7.593e-04,0.012,8.097e-05,0.047,2.220e-04
U6,0.003,-4.038e+00,0.003,0.002,0.085,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,7.593e-04,0.012,8.097e-05,0.047,2.220e-04
e-tron,0.003,5.947e-05,-7.533,0.002,0.085,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,7.593e-04,0.012,8.097e-05,0.047,2.220e-04
e-tron GT,0.003,5.947e-05,0.003,-13.651,0.085,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,7.593e-04,0.012,8.097e-05,0.047,2.220e-04
Q4 e-tron,0.003,5.947e-05,0.003,0.002,-7.289,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,7.593e-04,0.012,8.097e-05,0.047,2.220e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.003,5.947e-05,0.003,0.002,0.085,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,-1.957e+00,0.012,8.097e-05,0.047,2.220e-04
C40,0.003,5.947e-05,0.003,0.002,0.085,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,7.593e-04,-4.810,8.097e-05,0.047,2.220e-04
EX30,0.003,5.947e-05,0.003,0.002,0.085,0.02,1.742e-05,0.053,0.015,0.004,0.022,0.026,0.026,0.004,0.003,9.736e-05,5.640e-04,1.540e-04,4.672e-04,0.003,0.003,0.001,1.573e-04,0.024,7.867e-04,...,0.011,0.021,4.189e-04,0.073,3.471e-05,6.273e-05,0.001,0.083,0.007,0.004,0.4,0.022,1.622e-04,4.570e-04,1.718e-05,0.022,0.059,0.023,0.005,0.035,7.593e-04,0.012,-4.123e+00,0.047,2.220e-04


# Nested Logit

In [358]:
def nested_logit(df):
    groups = df.groupby(['market_ids', 'nesting_ids'])
    df['demand_instruments20'] = groups['shares'].transform(np.size)
    nl_formulation = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)+C(market_ids)')
    problem = pyblp.Problem(nl_formulation, df)
    return problem.solve(rho=0.5) # rho is initial value at which the optimization routine will start

In [359]:
nl_results = nested_logit(product_data)
nl_results

Problem Results Summary:
GMM   Objective    Projected     Reduced   Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm   Hessian   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------  -------  ----------------  -----------------
 2    +5.52E+00    +5.52E-12    +1.40E+01     0        +1.08E+06          +5.55E+03    

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective 
   Time      Converged   Iterations   Evaluations
-----------  ---------  ------------  -----------
 00:00:00       Yes          2             8     

Rho Estimates (Robust SEs in Parentheses):
All Groups 
-----------
 +3.44E-01 
(+3.88E-01)

Beta Estimates (Robust SEs in Parentheses):
  prices        Range         HP       Chargetime 
-----------  -----------  -----------  -----------
 -8.87E-02    +1.37E-01    +5.65E-02    -2.50E-02 
(+3.03E-02)  (+3.19E-02)  (+3.38E-02)  (+1.21E-02)

## Nested Logit GH Price-Elasticities

In [360]:
nl_elasticities= nl_results.compute_elasticities(market_id=2023) # name = 'Range' for elasticities of Range
pd.DataFrame(nl_elasticities, index = data2023['product_ids'], columns=data2023['product_ids'])

product_ids,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,iX,iX1,iX3,Atto 3,Dolphin,Han,Seal,Tang,Berlingo,C4,ë-C4 X,e-SpaceTourer,Jumpy,Born,Spring,...,Taycan,Megane,Zoe,Enyaq iV,Fortwo,Korando,Solterra,Model 3,Model S,Model X,Model Y,bZ4X,Proace City Verso,Proace Verso,Golf,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
product_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
U5,-3.565,2.565e-04,0.002,0.001,0.367,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,0.053,6.416e-05,0.205,1.759e-04
U6,0.011,-4.875e+00,0.002,0.001,0.367,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,0.053,6.416e-05,0.205,1.759e-04
e-tron,0.002,4.713e-05,-8.817,0.173,0.067,0.016,1.380e-05,0.042,0.012,0.482,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,6.234e-04,...,1.200,0.017,3.320e-04,0.058,2.750e-05,4.971e-05,8.234e-04,0.066,0.780,0.459,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,6.017e-04,0.010,6.416e-05,0.038,1.759e-04
e-tron GT,0.002,4.713e-05,0.282,-16.312,0.067,0.016,1.380e-05,0.042,0.012,0.482,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,6.234e-04,...,1.200,0.017,3.320e-04,0.058,2.750e-05,4.971e-05,8.234e-04,0.066,0.780,0.459,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,6.017e-04,0.010,6.416e-05,0.038,1.759e-04
Q4 e-tron,0.011,2.565e-04,0.002,0.001,-8.536,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,0.053,6.416e-05,0.205,1.759e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.002,4.713e-05,0.002,0.001,0.067,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,3.660e-01,...,0.008,0.017,3.320e-04,0.058,1.615e-02,4.971e-05,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,-2.011e+00,0.010,6.416e-05,0.038,1.759e-04
C40,0.011,2.565e-04,0.002,0.001,0.367,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,-5.769,6.416e-05,0.205,1.759e-04
EX30,0.002,4.713e-05,0.002,0.001,0.067,0.016,5.612e-04,0.042,0.012,0.003,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,6.234e-04,...,0.008,0.017,1.350e-02,0.058,2.750e-05,4.971e-05,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,6.017e-04,0.010,-4.976e+00,0.038,1.759e-04


### Inelastic demand

In [361]:
total_count = 0
for i in range(2013,2024):
    nl_elasticities1 = nl_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(nl_elasticities1).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 0


# Nested Logit GH probabilities, costs, profits, prices

In [362]:
product_data['CCP'] = nl_results.compute_probabilities()
product_data['costs'] = nl_results.compute_costs()
product_data['profit_per_car'] = product_data['prices'] - product_data['costs']
product_data['markups'] = product_data['profit_per_car'] / product_data['costs']
product_data['profits'] = nl_results.compute_profits()
product_data['eq_prices'] = nl_results.compute_prices()
product_data[['prices', 'CCP', 'costs', 'profit_per_car', 'markups', 'profits', 'eq_prices']].describe()

,prices,CCP,costs,profit_per_car,markups,profits,eq_prices
count,334.000,3.340e+02,334.000,334.000,334.000,3.340e+02,334.000
mean,46.911,2.128e-03,38.734,8.178,0.346,1.868e-02,46.911
std,29.975,5.618e-03,29.870,1.087,0.319,5.915e-02,29.975
min,12.486,4.347e-06,3.612,7.398,0.040,3.216e-05,12.486
25%,28.398,1.227e-04,20.797,7.478,0.174,9.605e-04,28.398
50%,35.904,7.149e-04,28.034,7.707,0.281,5.795e-03,35.904
75%,52.962,2.161e-03,45.216,8.397,0.373,1.808e-02,52.962
max,194.052,8.814e-02,186.577,11.421,2.457,9.689e-01,194.052


In [363]:
product_data.loc[product_data['market_ids'] == 2023]

,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,nesting_ids,Country,Sales,Intercept,Range_BLP,HP_BLP,Chargetime_BLP,Range_GH,HP_GH,Chargetime_GH,China,outside_share,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments20,CCP,costs,profit_per_car,markups,profits,eq_prices
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,41774,26403,3241,27923,15010,2099,1,0.696,27923,15010,2099,29,8.690e-04,19.032,7.420,0.390,6.448e-03,26.452
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,41769,26390,3241,27468,14562,2099,1,0.696,27468,14562,2099,29,1.473e-05,28.643,7.420,0.259,1.093e-04,36.064
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,41799,26202,3258,31201,19786,3157,0,0.696,31201,19786,3157,11,3.338e-04,59.514,7.790,0.131,2.601e-03,67.304
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,41702,26082,3258,21894,21035,3157,0,0.696,21894,21035,3157,11,1.129e-04,114.144,7.790,0.068,8.796e-04,121.934
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,41678,26323,3247,23156,17256,1813,0,0.696,23156,17256,1813,29,1.153e-02,58.126,7.729,0.133,8.913e-02,65.855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,41918,26523,3227,37374,22690,2733,0,0.696,37374,22690,2733,3,3.878e-04,8.594,8.892,1.035,3.449e-03,17.485
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,41708,26202,3247,22124,19786,1813,0,0.696,22124,19786,1813,29,2.538e-03,35.333,7.734,0.219,1.963e-02,43.066
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,41699,26336,3247,21894,17776,1813,0,0.696,21894,17776,1813,16,1.964e-05,29.362,7.463,0.254,1.466e-04,36.825
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,41717,26202,3247,22067,19786,1813,0,0.696,22067,19786,1813,29,9.647e-03,36.193,7.734,0.214,7.461e-02,43.927


# Cost Side

In [364]:
statsmodels_ols = smf.ols('np.log(costs) ~ 1 + Range + HP + Chargetime', product_data)
statsmodels_results = statsmodels_ols.fit(cov_type='HC0')
statsmodels_results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.650   
Dependent Variable: np.log(costs)    AIC:                347.0854
Date:               2024-06-05 01:31 BIC:                362.3299
No. Observations:   334              Log-Likelihood:     -169.54 
Df Model:           3                F-statistic:        172.6   
Df Residuals:       330              Prob (F-statistic): 2.76e-67
R-squared:          0.653            Scale:              0.16356 
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept      2.7511    0.1165  23.6120  0.0000   2.5228   2.9795
Range          0.0128    0.0028   4.6222  0.0000   0.0074   0.0182
HP             0.0237    0.0017  13.9533  0.0000   0.0204   0.0271
Chargetime    -0.0128    0.0023  -5.5617  0.0000  -0.0173  -0.0083
-----------------------------------------------------------------
Omnibus:              7.667        Durbin-Watson:           0.611
Prob(Omnibus):        0.022        Jarque-Bera (JB):        9.667
Skew:                 0.206        Prob(JB):                0.008
Kurtosis:             3.725        Condition No.:           333  
=================================================================

"""

# Graphs

In [365]:
anlysis_data = product_data[product_data['market_ids'] > 2019]

In [366]:
biggest_firms = anlysis_data.groupby('firm_ids')['shares'].sum().nlargest(5)
biggest_firms

firm_ids
Tesla            0.162
Volkswagen       0.094
Audi             0.038
Mercedes-Benz    0.036
BMW              0.036
Name: shares, dtype: float64

# Counterfactual 1
#### China increases subsidy -> decreased price on the demand

In [367]:
year_2023 = 2023
cf1 = product_data.loc[product_data['market_ids'] == year_2023]
cf1

,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,nesting_ids,Country,Sales,Intercept,Range_BLP,HP_BLP,Chargetime_BLP,Range_GH,HP_GH,Chargetime_GH,China,outside_share,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments20,CCP,costs,profit_per_car,markups,profits,eq_prices
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,41774,26403,3241,27923,15010,2099,1,0.696,27923,15010,2099,29,8.690e-04,19.032,7.420,0.390,6.448e-03,26.452
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,41769,26390,3241,27468,14562,2099,1,0.696,27468,14562,2099,29,1.473e-05,28.643,7.420,0.259,1.093e-04,36.064
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,41799,26202,3258,31201,19786,3157,0,0.696,31201,19786,3157,11,3.338e-04,59.514,7.790,0.131,2.601e-03,67.304
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,41702,26082,3258,21894,21035,3157,0,0.696,21894,21035,3157,11,1.129e-04,114.144,7.790,0.068,8.796e-04,121.934
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,41678,26323,3247,23156,17256,1813,0,0.696,23156,17256,1813,29,1.153e-02,58.126,7.729,0.133,8.913e-02,65.855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,41918,26523,3227,37374,22690,2733,0,0.696,37374,22690,2733,3,3.878e-04,8.594,8.892,1.035,3.449e-03,17.485
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,41708,26202,3247,22124,19786,1813,0,0.696,22124,19786,1813,29,2.538e-03,35.333,7.734,0.219,1.963e-02,43.066
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,41699,26336,3247,21894,17776,1813,0,0.696,21894,17776,1813,16,1.964e-05,29.362,7.463,0.254,1.466e-04,36.825
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,41717,26202,3247,22067,19786,1813,0,0.696,22067,19786,1813,29,9.647e-03,36.193,7.734,0.214,7.461e-02,43.927


In [368]:
cf1['new_prices'] = cf1['prices']
cf1.loc[cf1['China'] == 1, 'new_prices'] *= 0.8
cf1['new_shares'] = nl_results.compute_shares(market_id=year_2023, prices=cf1['new_prices'])
cf1['iv_change'] = 100 * (cf1['new_shares'] - cf1['shares']) / cf1['shares']
cf1

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_10293/3474236962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf1['new_prices'] = cf1['prices']
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_10293/3474236962.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf1['new_shares'] = nl_results.compute_shares(market_id=year_2023, prices=cf1['new_prices'])
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_10293/3474236962.py:4: SettingWithCopyWarning: 
A value is trying to b

,ID,market_ids,shares,firm_ids,product_ids,Range,prices,HP,Chargetime,Type,nesting_ids,Country,Sales,Intercept,Range_BLP,HP_BLP,Chargetime_BLP,Range_GH,HP_GH,Chargetime_GH,China,outside_share,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments20,CCP,costs,profit_per_car,markups,profits,eq_prices,new_prices,new_shares,iv_change
2,1,2023,8.690e-04,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1.0,41774,26403,3241,27923,15010,2099,1,0.696,27923,15010,2099,29,8.690e-04,19.032,7.420,0.390,6.448e-03,26.452,21.162,1.718e-03,97.672
3,2,2023,1.473e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1.0,41769,26390,3241,27468,14562,2099,1,0.696,27468,14562,2099,29,1.473e-05,28.643,7.420,0.259,1.093e-04,36.064,28.851,3.775e-05,156.335
8,3,2023,3.338e-04,Audi,e-tron,37.5,67.304,40.2,17,SUV,F,DE,68,1.0,41799,26202,3258,31201,19786,3157,0,0.696,31201,19786,3157,11,3.338e-04,59.514,7.790,0.131,2.601e-03,67.304,67.304,2.873e-04,-13.933
11,4,2023,1.129e-04,Audi,e-tron GT,47.2,121.934,52.2,17,Sedan,F,DE,23,1.0,41702,26082,3258,21894,21035,3157,0,0.696,21894,21035,3157,11,1.129e-04,114.144,7.790,0.068,8.796e-04,121.934,121.934,9.718e-05,-13.933
14,5,2023,1.153e-02,Audi,Q4 e-tron,49.6,65.855,28.1,28,SUV,C,DE,2349,1.0,41678,26323,3247,23156,17256,1813,0,0.696,23156,17256,1813,29,1.153e-02,58.126,7.729,0.133,8.913e-02,65.855,65.855,1.115e-02,-3.321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,187,2023,3.878e-04,Volkswagen,up!,25.6,17.485,8.1,48,Hatchback,A,DE,79,1.0,41918,26523,3227,37374,22690,2733,0,0.696,37374,22690,2733,3,3.878e-04,8.594,8.892,1.035,3.449e-03,17.485,17.485,3.842e-04,-0.951
328,188,2023,2.538e-03,Volvo,C40,46.6,43.066,40.2,28,SUV,C,SE,517,1.0,41708,26202,3247,22124,19786,1813,0,0.696,22124,19786,1813,29,2.538e-03,35.333,7.734,0.219,1.963e-02,43.066,43.066,2.454e-03,-3.321
329,189,2023,1.964e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,1.0,41699,26336,3247,21894,17776,1813,0,0.696,21894,17776,1813,16,1.964e-05,29.362,7.463,0.254,1.466e-04,36.825,36.825,1.906e-05,-2.921
332,190,2023,9.647e-03,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,1.0,41717,26202,3247,22067,19786,1813,0,0.696,22067,19786,1813,29,9.647e-03,36.193,7.734,0.214,7.461e-02,43.927,43.927,9.327e-03,-3.321
